# Estandarizar los datos de la base de publicidad y QuienEsQuien
## NO corras este notebook excepto que quieras que se tarde una eternidad!

Generar un diccionario de beneficiarios de los contratos de publicidad a partir de la base de Comunicación Social de la base completa de  http://publicidadoficial.com.mx/ .
Utilizamos fuzzy wuzzy para estandarizar los nombres y así poder matchear los contratos con una sola entidad cuando se trate de la misma. 


In [1]:
import pandas as pd
from utils import *
from pickle import dump

In [2]:
sexenios = ['FOX','CAL','EPN']
colors_sex = {'FOX':'red','CAL':'green','EPN':'blue'}

Cargar datos

In [18]:
datos = pd.read_excel('data-raw/base-completa-comsoc-2013-2018.xlsx', usecols=[19])
datos = datos['Beneficiario'].unique().tolist()
datos = [clean_string(s) for s in datos]
datos = list(set(datos))
datos.sort()
len(datos), datos[0:10]

(3392,
 ['2 Realpeople Solutions Sa De Cv',
  '35 Am S De Rl De Cv',
  '360 Led Mexico S De Rl De Cv',
  '360 Media Sa De Cv',
  '365 Consulting Sc',
  '3D Estudio Sa De Cv',
  '3Difica Sa De Cv',
  '5M2 Andenes Sapi De Cv',
  '5M2 Digital Sapi De Cv',
  '5M2 Sa De Cv'])

In [27]:
catalogo = [pd.read_csv('data-clean/{}-empresa.csv'.format(s)) for s in sexenios]
catalogo = [df['empresa'].dropna().unique().tolist() for df in catalogo]
catalogo = [item for sublist in catalogo for item in sublist]
catalogo = list(set(catalogo))
catalogo.sort()
len(catalogo), catalogo[0:10]


(190642,
 ['029 Grupo Constructor Y De Desarrollo Sa De Cv',
  '029 Grupo Constructor Y De Desarrollosa De Cv',
  '0Donnell Rivera Natalie Marie',
  '0Ra Flor Ramirez Calderon',
  '0Ra Patricia Alvarez Aguila',
  '1 2 3 Hazlo Sa De Cv',
  '1 A H C Producciones S A De C V',
  '1 A H C Producciones Sa De Cv',
  '1 Ahc Producciones Sa De Cv',
  '1 Leycon Grupo Constructor Sa De Cv'])

In [30]:
print("Empresas en ambos datasets:", len(set(datos).intersection(set(catalogo))))

Empresas en ambos datasets: 1256


correr fuzzy wuzzy

In [6]:
dic = generar_diccionario_mapeo(datos, catalogo, warn=95, limit=90)
len(dic)

/home/esther/Dropbox/AnalisisDatos/Publiton2019/utils.py:79: UserWarning: 
	BAJA SIMILITUD: jose luis garcia rebollo.
	Opciones: jose luis garcia tello (91), jose tamayo ruiz (86), grupo constructor santiago garcia sa de cv (86)
  warnings.warn( warn_message )
/home/esther/Dropbox/AnalisisDatos/Publiton2019/utils.py:79: UserWarning: 
	BAJA SIMILITUD: aeroimagen sa de cv.
	Opciones: ecoimagen sa de cv (92), farmagen sa de cv (89), ofimagen sa de cv (89)
  warnings.warn( warn_message )
/home/esther/Dropbox/AnalisisDatos/Publiton2019/utils.py:79: UserWarning: 
	IGNORADO: cinabrio ti sc.
	Opciones: u servers comunicaciones sc (86), instituto para el mejoramiento de la calidad de la inversion sc (86), consultores corporativos grupo sigma sc (86)
  warnings.warn( warn_message )
/home/esther/Dropbox/AnalisisDatos/Publiton2019/utils.py:79: UserWarning: 
	BAJA SIMILITUD: comunicacion y desarrollo editorial especializado sa de cv.
	Opciones: codeesa comunicacion y desarrollo editorial especializ

3392

In [32]:
print("Datos detectados manuealmente por fuzzy wuzzy:", len([1 for key,value in dic.items() if key!=value]))

Datos detectados manuealmente por fuzzy wuzzy: 946


Los warnings se revisan manualmente

Corregir los que se ignoraron inadecuadamente

In [15]:
with open("data-clean/dic-Publicidad2QuienEsQuien-ignorado.txt") as f:
    for line in f.readlines():
        if line.startswith('IGNORADO: '):
            key = line.replace('IGNORADO: ','').replace('.','').strip()
        if line.startswith('Opciones: '):
            value = line.replace('Opciones: ','')
            value = value.split('(')[0].strip()
            dic[key] = value

Corregir los que se cambiaron inadecuadamente

In [16]:
with open("data-clean/dic-Publicidad2QuienEsQuien-similitud.txt") as f:
    for line in f.readlines():
        if line.startswith('-'):
            key = line.replace('-BAJA SIMILITUD: ','').replace('.','').strip()
            dic[key] = key

Revisar y guardar

In [36]:
print("Empresas totales: ", len( set(datos) ))

Empresas totales:  3392


In [37]:
print("Correcion fuzzywuzzy + manual:", len([1 for key,value in dic.items() if key!=value]))

Correcion fuzzywuzzy + manual: 946


In [34]:
print("Empresas en ambos datasets:", len(set(datos).intersection(set(dic.values()))))

Empresas en ambos datasets: 2459


In [38]:
pickle.dump( dic, open( "data-clean/dic-Publicidad2QuienEsQuien.p", "wb" ) )